<a href="https://colab.research.google.com/github/DoItSon/playdata/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%ED%95%B4%EB%B3%B4%EA%B8%B0!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 메모장
- 0.8189379090413184 = 0.5
- 0.8236173245493615 = 0.4
- 0.8236625863676823 = 0.3
- 0.8256464235755943 = 0.35

- 0.823185924948775 머신러닝 결측치
- 0.8247486240372479 내 결측치

- 추후 베이지안 최저화를 통해 최적의 파라미터 찾기!

In [727]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import random

import warnings
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [728]:
# !pip install optuna

In [729]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [730]:
DATA_PATH = "/content/drive/MyDrive/"

In [731]:
train = pd.read_csv(f'{DATA_PATH}train.csv')
test = pd.read_csv(f'{DATA_PATH}test.csv')

In [732]:
categorical_features_enc = ['COMPONENT_ARBITRARY_COMPONENT1',
       'COMPONENT_ARBITRARY_COMPONENT2', 'COMPONENT_ARBITRARY_COMPONENT3',
       'COMPONENT_ARBITRARY_COMPONENT4', 'YEAR_2007', 'YEAR_2008', 'YEAR_2009',
       'YEAR_2010', 'YEAR_2011', 'YEAR_2012', 'YEAR_2013', 'YEAR_2014',
       'YEAR_2015', 'YEAR_2016', 'YEAR_2017', 'YEAR_2018', 'YEAR_2019',
       'YEAR_2020', 'YEAR_2021', 'YEAR_2022']
# Inference(실제 진단 환경)에 사용하는 컬럼
test_stage_features_enc = ['ANONYMOUS_1', 'SAMPLE_TRANSFER_DAY', 'ANONYMOUS_2', 'AG', 'AL', 'B',
       'BA', 'BE', 'CA', 'CO', 'CR', 'CU', 'FE', 'H2O', 'LI', 'MG', 'MN', 'MO',
       'NA', 'NI', 'P', 'PB', 'PQINDEX', 'S', 'SB', 'SI', 'SN', 'TI', 'V',
       'V40', 'ZN', 'CD', 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FOXID', 'FSO4',
       'FTBN', 'FUEL', 'K', 'SOOTPERCENTAGE', 'U100', 'U75', 'U50', 'U25',
       'U20', 'U14', 'U6', 'U4', 'V100', 'COMPONENT_ARBITRARY_COMPONENT1',
       'COMPONENT_ARBITRARY_COMPONENT2', 'COMPONENT_ARBITRARY_COMPONENT3',
       'COMPONENT_ARBITRARY_COMPONENT4', 'YEAR_2007', 'YEAR_2008', 'YEAR_2009',
       'YEAR_2010', 'YEAR_2011', 'YEAR_2012', 'YEAR_2013', 'YEAR_2014',
       'YEAR_2015', 'YEAR_2016', 'YEAR_2017', 'YEAR_2018', 'YEAR_2019',
       'YEAR_2020', 'YEAR_2021', 'YEAR_2022']

## 전처리

In [733]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectPercentile

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [734]:
SEED = 42

- 내가 직접한 결측치

In [735]:
# fill_mean_func = lambda x: x.fillna(x.median())
# zx = train.groupby(["SAMPLE_TRANSFER_DAY","COMPONENT_ARBITRARY"]).apply(fill_mean_func)
# xc = zx.groupby(["ANONYMOUS_2","YEAR"]).apply(fill_mean_func)

# xc1 = xc.groupby(["ANONYMOUS_1","ANONYMOUS_2"]).apply(fill_mean_func)
# feat = pd.DataFrame(train["ID"].unique(),columns = ["ID"])

# xc2 = xc1.groupby(["MO"]).apply(fill_mean_func)
# features = feat.merge(xc2,how="left",on="ID")

# train = features

In [736]:
# train.isnull().sum()

- 머신러닝 결측치

In [737]:
# col_lst2 = ["CD","FH2O","FNOX","FOPTIMETHGLY","FOXID","FSO4","FTBN","FUEL","K","SOOTPERCENTAGE","U100","U75","U50","U25","U20","U14","U6","U4","V100"]

In [738]:
# imputer = IterativeImputer(estimator =  GradientBoostingRegressor(),random_state=42)
# tmp = imputer.fit_transform(train[col_lst2])

In [740]:
# x2 = pd.DataFrame(tmp,columns=col_lst2)
# train = train.drop(columns=col_lst2)
# train = pd.concat([train,x2],axis=1)
# train

In [743]:
train = train.fillna(0)
test = test.fillna(0)

In [ ]:
train.isnull().sum().sum()

## 데이터 분리

In [744]:
train_X  = train.drop(['ID', 'Y_LABEL'], axis = 1) # 정답값 제외
target  = train['Y_LABEL'] # 정답

test = test.drop(['ID'], axis = 1) # 추론 데이터

In [745]:
train_X

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,...,U50,U25,U20,U14,U6,U4,V,V100,V40,ZN
0,COMPONENT3,1486,2011,7,200,0,3,93,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,154.0,75
1,COMPONENT2,1350,2021,51,375,0,2,19,0,0,...,1.0,2.0,4.0,6.0,216.0,1454.0,0,0.0,44.0,652
2,COMPONENT2,2415,2015,2,200,0,110,1,1,0,...,0.0,0.0,3.0,39.0,11261.0,41081.0,0,0.0,72.6,412
3,COMPONENT3,7389,2010,2,200,0,8,3,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,133.3,7
4,COMPONENT3,3954,2015,4,200,0,1,157,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,133.1,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,COMPONENT3,1616,2014,8,200,0,2,201,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,135.4,16
14091,COMPONENT1,2784,2013,2,200,0,3,85,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,14.5,117.5,1408
14092,COMPONENT3,1788,2008,9,550,0,6,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,54.0,1301
14093,COMPONENT2,2498,2009,19,550,0,2,4,0,0,...,0.0,7.0,8.0,100.0,1625.0,18890.0,0,0.0,44.3,652


# 원핫인코딩

In [746]:
enc = OneHotEncoder()

tmp = pd.DataFrame(
    enc.fit_transform(train_X[["COMPONENT_ARBITRARY",'YEAR']]).toarray(),
    columns = enc.get_feature_names_out()
)
train_X = pd.concat([train_X,tmp],axis=1).drop(columns = ["COMPONENT_ARBITRARY",'YEAR'])

tmp1 = pd.DataFrame(
    enc.transform(test[["COMPONENT_ARBITRARY",'YEAR']]).toarray(),
    columns = enc.get_feature_names_out()
)
test = pd.concat([test,tmp1],axis=1).drop(columns = ["COMPONENT_ARBITRARY",'YEAR'])

In [747]:
train_X.shape,target.shape,test.shape

((14095, 70), (14095,), (6041, 36))

# 스케일링

In [749]:
def get_values(value):
    return value.values.reshape(-1, 1)

for col in train_X.columns:
    if col not in categorical_features_enc:
        scaler = StandardScaler()
        train_X[col] = scaler.fit_transform(get_values(train_X[col]))
        if col in test.columns:
            test[col] = scaler.transform(get_values(test[col]))

## Classifier

In [750]:
import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import optuna

In [825]:
def objective_1(trial):
    hp = {
        "max_depth" : trial.suggest_int("max_depth",2,15), # 찾을 범위의 최소값과 최대값
        "min_samples_split" : trial.suggest_int("min_samples_split",2,10),
        "criterion" : trial.suggest_categorical("criterion",["gini","entropy"]),
        "max_leaf_nodes" : trial.suggest_int("max_leaf_nodes",5,10),
        "n_estimators" : trial.suggest_int("n_estimators",10,500),
        "learning_rate" : trial.suggest_float("learning_rate",0.01,0.1)
    }
    cv = KFold(n_splits=5,shuffle=True,random_state=SEED) # SEED 함수내부에서 전역변수 사용 (이렇게하면 안됨!)
    model = LGBMClassifier(random_state=SEED,**hp)

    scores = cross_val_score(model, train_X, target, cv = cv, scoring="f1_macro") 
    return np.mean(scores)

In [826]:
sampler = optuna.samplers.TPESampler(seed = SEED) 

study_1 = optuna.create_study(
    direction= "maximize", 
    sampler = sampler
)

study_1.optimize(objective_1,n_trials=50)

[I 2022-11-22 07:18:59,467] A new study created in memory with name: no-name-30f038c1-50e0-446d-89ee-6a77097be5ce
[I 2022-11-22 07:19:03,789] Trial 0 finished with value: 0.8100299654745882 and parameters: {'max_depth': 7, 'min_samples_split': 10, 'criterion': 'gini', 'max_leaf_nodes': 5, 'n_estimators': 86, 'learning_rate': 0.015227525095137952}. Best is trial 0 with value: 0.8100299654745882.
[I 2022-11-22 07:19:25,204] Trial 1 finished with value: 0.8199376913720652 and parameters: {'max_depth': 14, 'min_samples_split': 7, 'criterion': 'gini', 'max_leaf_nodes': 10, 'n_estimators': 418, 'learning_rate': 0.029110519961044856}. Best is trial 1 with value: 0.8199376913720652.
[I 2022-11-22 07:19:29,206] Trial 2 finished with value: 0.8183271443863432 and parameters: {'max_depth': 4, 'min_samples_split': 3, 'criterion': 'entropy', 'max_leaf_nodes': 7, 'n_estimators': 152, 'learning_rate': 0.06506676052501416}. Best is trial 1 with value: 0.8199376913720652.
[I 2022-11-22 07:19:31,813] Tr

In [827]:
print("Best Score:",study_1.best_value) 
print("Best hp", study_1.best_params)

Best Score: 0.823099389064776
Best hp {'max_depth': 8, 'min_samples_split': 7, 'criterion': 'gini', 'max_leaf_nodes': 9, 'n_estimators': 495, 'learning_rate': 0.07171518532882763}


In [828]:
study_1.best_params

{'max_depth': 8,
 'min_samples_split': 7,
 'criterion': 'gini',
 'max_leaf_nodes': 9,
 'n_estimators': 495,
 'learning_rate': 0.07171518532882763}

In [829]:
patience = 50
cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

In [ ]:
pred_list = []
for tri,vai in cv.split(train_X):
    # 학습데이터
    x_train = train_X.iloc[tri]
    y_train = target.iloc[tri]
    # 검증데이터
    x_valid = train_X.iloc[vai]
    y_valid = target.iloc[vai]
    
    model = LGBMClassifier(random_state=SEED,**study_1.best_params)
    model.fit(x_train,y_train , eval_set = [(x_train,y_train)],early_stopping_rounds=50) 

    pred = model.predict_proba(x_valid)[:,1] # 1일 확률
    pred_list.append(pred)

In [768]:
pred_list

[array([3.85768355e-08, 4.48041497e-05, 1.46039312e-07, ...,
        3.41770225e-08, 1.69695185e-08, 2.14449093e-04]),
 array([2.18955412e-08, 5.78213455e-04, 1.49992617e-05, ...,
        2.26793575e-09, 7.00864425e-08, 3.37829388e-06]),
 array([8.18020379e-06, 1.78788616e-07, 7.88823240e-04, ...,
        1.63301935e-06, 1.45102756e-08, 4.62784335e-07]),
 array([9.99999991e-01, 1.27270025e-07, 2.71990793e-08, ...,
        9.98856918e-01, 1.38410732e-04, 6.32453215e-08]),
 array([1.55261320e-06, 4.31170031e-06, 5.20739363e-06, ...,
        4.51630279e-06, 1.36228980e-06, 4.39584427e-01])]

In [831]:
pred = np.concatenate(pred_list,axis=0)
pred.shape

(14095,)

In [832]:
pred

array([0.00144789, 0.01908173, 0.00166618, ..., 0.0023883 , 0.00079507,
       0.3590306 ])

In [833]:
# score_list = []
    
# # 모델 학습
# model = LGBMClassifier(random_state=SEED,n_estimators=1000)
# model.fit(train_X,target , eval_set = [(train_X,target)],early_stopping_rounds=50) 

# pred = model.predict_proba(train_X)[:,1] # 1일 확률
# # predict = np.where(pred > 0.35, 1 , 0)
# # score = f1_score(y_valid,predict,average="macro") 
# # score_list.append(score) 

# # score_list

- 확률 추출!

In [834]:
x_target = pd.DataFrame(pred)
x_target = x_target.squeeze()
x_target

0        0.001448
1        0.019082
2        0.001666
3        0.003031
4        0.000225
           ...   
14090    0.109547
14091    0.000628
14092    0.002388
14093    0.000795
14094    0.359031
Name: 0, Length: 14095, dtype: float64

## Regressor

- train의 부분 X의 예측값을 설명하는 Regressor

In [835]:
test_stage_features = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN',
       'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN',
       'COMPONENT_ARBITRARY_COMPONENT1', 'COMPONENT_ARBITRARY_COMPONENT2',
       'COMPONENT_ARBITRARY_COMPONENT3', 'COMPONENT_ARBITRARY_COMPONENT4',
       'YEAR_2007', 'YEAR_2008', 'YEAR_2009', 'YEAR_2010', 'YEAR_2011',
       'YEAR_2012', 'YEAR_2013', 'YEAR_2014', 'YEAR_2015', 'YEAR_2016',
       'YEAR_2017', 'YEAR_2018', 'YEAR_2019', 'YEAR_2020', 'YEAR_2021',
       'YEAR_2022']

In [836]:
train_reg = train_X[test_stage_features]
train_reg.shape,x_target.shape

((14095, 36), (14095,))

In [837]:
import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
import optuna
from sklearn.metrics import accuracy_score
from sklearn.metrics import explained_variance_score

In [838]:
patience = 50
cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

In [839]:
pred_list = []
for tri,vai in cv.split(train_reg):
    # 학습데이터
    x_train = train_reg.iloc[tri]
    y_train = x_target.iloc[tri]
    # 검증데이터
    x_valid = train_reg.iloc[vai]
    y_valid = x_target.iloc[vai]
    
    # 모델 학습
    model = lgb.LGBMRegressor(random_state=SEED,n_estimators=1000)
    model.fit(x_train,y_train , eval_set = [(x_valid,y_valid)],early_stopping_rounds=50) 

    pred = model.predict(x_valid)

    predict = np.where(pred > 0.35, 1 , 0)
    # score = cross_val_score(y_valid, predict, scoring="f1_macro") 
    pred_list.append(pred)

pred_list

[1]	valid_0's l2: 0.0401544
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's l2: 0.0401659
[3]	valid_0's l2: 0.0401463
[4]	valid_0's l2: 0.0401691
[5]	valid_0's l2: 0.0402011
[6]	valid_0's l2: 0.0402088
[7]	valid_0's l2: 0.0402187
[8]	valid_0's l2: 0.0402476
[9]	valid_0's l2: 0.0402208
[10]	valid_0's l2: 0.0402354
[11]	valid_0's l2: 0.0402086
[12]	valid_0's l2: 0.0402241
[13]	valid_0's l2: 0.0402298
[14]	valid_0's l2: 0.0402424
[15]	valid_0's l2: 0.0402851
[16]	valid_0's l2: 0.0403145
[17]	valid_0's l2: 0.0403106
[18]	valid_0's l2: 0.0403105
[19]	valid_0's l2: 0.0403378
[20]	valid_0's l2: 0.0403321
[21]	valid_0's l2: 0.0403557
[22]	valid_0's l2: 0.0403698
[23]	valid_0's l2: 0.0403418
[24]	valid_0's l2: 0.0403453
[25]	valid_0's l2: 0.0403339
[26]	valid_0's l2: 0.0403201
[27]	valid_0's l2: 0.0403454
[28]	valid_0's l2: 0.0403878
[29]	valid_0's l2: 0.0403991
[30]	valid_0's l2: 0.0403961
[31]	valid_0's l2: 0.0403968
[32]	valid_0's l2: 0.0403965
[33]	valid_0's l2: 

[array([0.05547893, 0.05614098, 0.05734411, ..., 0.05870844, 0.05308926,
        0.06012224]),
 array([0.04677611, 0.0466501 , 0.05157413, ..., 0.05261186, 0.05540728,
        0.06433886]),
 array([0.05616909, 0.0580924 , 0.05525765, ..., 0.05616909, 0.05597911,
        0.05745312]),
 array([0.05806057, 0.05806057, 0.05806057, ..., 0.05806057, 0.05806057,
        0.05806057]),
 array([0.05696184, 0.0560256 , 0.05241345, ..., 0.05417779, 0.05545542,
        0.05377298])]

In [840]:
pred = np.concatenate(pred_list,axis=0)
pred.shape

(14095,)

In [841]:
pred # 이것이 예측값을 설명한다는 게 무슨 의미지?

array([0.05547893, 0.05614098, 0.05734411, ..., 0.05417779, 0.05545542,
       0.05377298])

### test와 shape을 맞춘 후 test에 적합

In [842]:
re_x_target = x_target.drop(index=range(6041,14095))
test.shape, re_x_target.shape

((6041, 36), (6041,))

In [843]:
pred_list = []
for tri,vai in cv.split(test):
    # 학습데이터
    x_train = test.iloc[tri]
    y_train = re_x_target.iloc[tri]
    # 검증데이터
    x_valid = test.iloc[vai]
    y_valid = re_x_target.iloc[vai]
    
    # 모델 학습
    model = lgb.LGBMRegressor(random_state=SEED,n_estimators=1000)
    model.fit(x_train,y_train , eval_set = [(x_valid,y_valid)],early_stopping_rounds=50) 

    pred = model.predict(x_valid)

    predict = np.where(pred > 0.35, 1 , 0)
    # score = cross_val_score(y_valid, predict, scoring="f1_macro") 
    pred_list.append(predict)

pred_list

[1]	valid_0's l2: 0.0358647
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's l2: 0.0359527
[3]	valid_0's l2: 0.036041
[4]	valid_0's l2: 0.036075
[5]	valid_0's l2: 0.0361908
[6]	valid_0's l2: 0.036242
[7]	valid_0's l2: 0.0363063
[8]	valid_0's l2: 0.0363149
[9]	valid_0's l2: 0.0364076
[10]	valid_0's l2: 0.0363994
[11]	valid_0's l2: 0.0364847
[12]	valid_0's l2: 0.0365233
[13]	valid_0's l2: 0.0366047
[14]	valid_0's l2: 0.0366467
[15]	valid_0's l2: 0.0366858
[16]	valid_0's l2: 0.0367271
[17]	valid_0's l2: 0.036703
[18]	valid_0's l2: 0.0367689
[19]	valid_0's l2: 0.0368063
[20]	valid_0's l2: 0.0368666
[21]	valid_0's l2: 0.0369441
[22]	valid_0's l2: 0.0369439
[23]	valid_0's l2: 0.0369572
[24]	valid_0's l2: 0.0370268
[25]	valid_0's l2: 0.0370445
[26]	valid_0's l2: 0.0370624
[27]	valid_0's l2: 0.0371214
[28]	valid_0's l2: 0.0371921
[29]	valid_0's l2: 0.0372289
[30]	valid_0's l2: 0.0372294
[31]	valid_0's l2: 0.0372279
[32]	valid_0's l2: 0.037233
[33]	valid_0's l2: 0.037

[array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0])]

In [844]:
x = np.concatenate(pred_list,axis = 0)
x.shape

(6041,)

In [845]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')
submit['Y_LABEL'] = x
submit.head()

,ID,Y_LABEL
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0


In [820]:
submit.to_csv('/content/drive/MyDrive/submit.csv', index=False)